# Corrective RAG

![corrective rag](https://pbs.twimg.com/media/GF1JIZ_aEAE1cZ1.jpg:large "100x100")

In [ ]:
!cat /etc/*-release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.3 LTS"
PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.3 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


# Setup

## Install python dependencies

In [ ]:
! pip install --quiet langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph tavily-python langchain-mistralai

## ![](https://ollama.com/public/ollama.png) Download and Install ollama



Visit [Ollama](https://ollama.com/) to dowload it manually.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!ollama serve & ollama pull mistral:instruct

## Set Environment secret variables

In [ ]:
import os

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = 'ls__91f5ff25d23a450990347fa2aeaf159f'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['TAVILY_API_KEY'] = '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'
mistral_api_key = "<Mistral-Key>" # if using

## Set model and other configs

In [ ]:
run_local = "Yes"
local_llm = "mistral:instruct"

## Setup Llama.cpp (Optional if using GPT4All)


GPT4All embeddings are much better than Llama.cpp embeddings as GPT4All embeddings retrieval are more richer.

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 19513, done.
remote: Counting objects: 100% (11195/11195), done.
remote: Compressing objects: 100% (543/543), done.
remote: Total 19513 (delta 10811), reused 10720 (delta 10651), pack-reused 8318
Receiving objects: 100% (19513/19513), 19.90 MiB | 22.72 MiB/s, done.
Resolving deltas: 100% (13938/13938), done.


In [ ]:
%cd llama.cpp
!make
%cd ../

/content/llama.cpp
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG 
I NVCCFLAGS: -std=c++11 -O3 
I LDFLAGS:    
I CC:        cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
I CXX:       g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0

cc  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -

### Download embeddings model

In [ ]:
!wget https://huggingface.co/nomic-ai/nomic-embed-text-v1-GGUF/resolve/main/nomic-embed-text-v1.Q2_K.gguf -P llama.cpp/model

--2024-03-01 06:12:59--  https://huggingface.co/nomic-ai/nomic-embed-text-v1-GGUF/resolve/main/nomic-embed-text-v1.Q2_K.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.4, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/a6/63/a663fb46c8f5bfbd779f9398e78efdd80ccde64ee2834e1b61ff49a34748575d/afb87e81c67d34db721db27f093d1e87e4620001fe11e4566c5ceb88cd0fc667?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27nomic-embed-text-v1.Q2_K.gguf%3B+filename%3D%22nomic-embed-text-v1.Q2_K.gguf%22%3B&Expires=1709532779&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTUzMjc3OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E2LzYzL2E2NjNmYjQ2YzhmNWJmYmQ3NzlmOTM5OGU3OGVmZGQ4MGNjZGU2NGVlMjgzNGUxYjYxZmY0OWEzNDc0ODU3NWQvYWZiODdlODFjNjdkMzRkYjcy

In [ ]:
!ls llama.cpp/model

nomic-embed-text-v1.Q2_K.gguf


### Install llama cpp python package

In [ ]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 7.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.53-cp310-cp310-manylinux_2_35_x86_64.whl size=2751939 sha256=f2ab2178b8ca803c67b8e7dffc787894b20b7bdb7984c0c719c4b742fe90887b
  Stored in directory: /root/.cache/pip/wheels/56/94/10/5407d0fd0a979d77bef8df3d298af7d1d60193b6b759ad217f
Successfully built llama-cpp-python


## Install GPT4ALL (if not using Llama.cpp)


> For using GPT4ALL Embeddings ubuntu-22 is required otherwise use LlamaCPP embeddings.


In [ ]:
!pip install gpt4all

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 18.1 MB/s eta 0:00:00


# Indexing

## Split documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

# Load url
url = "https://lilianweng.github.io/posts/2023-06-23-agent/"
loader = WebBaseLoader(url)
docs = loader.load()

# split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size= 500,
    chunk_overlap= 100
)
all_splits= text_splitter.split_documents(docs)

print(f"Total splits created: {len(all_splits)}")

Total splits created: 32


In [ ]:
# Lowest chunk in all_splits
print(f"Lowest split size: {min(len(d.page_content) for d in all_splits)}")
min_value = sorted([(i, d.page_content, len(d.page_content)) for i, d in enumerate(all_splits)],key= lambda x: x[2])[0]
print(min_value)

Lowest split size: 48
(25, 'Conversatin samples:\n[\n  {\n    "role": "system",', 48)


## Create retriever

In [ ]:
from langchain_community.embeddings import LlamaCppEmbeddings, GPT4AllEmbeddings

# Embed and index
if run_local == "Yes":
    # GPT4All
    embedding = GPT4AllEmbeddings()
    # embd_model_path = os.path.join(os.getcwd(), "llama.cpp", "model", "nomic-embed-text-v1.Q2_K.gguf")
    # embedding = LlamaCppEmbeddings(model_path=embd_model_path, n_batch=512)
else:
    pass

# Index
vectorstore = Chroma.from_documents(
    documents= all_splits,
    collection_name= "rag-chroma-1.0",
    embedding= embedding,
)

retriever= vectorstore.as_retriever()

100%|██████████| 45.9M/45.9M [00:00<00:00, 135MiB/s]


## Llama.cpp retriever (optional)

This retriever is optional as GPT4ALL embeddings are better than Llama.cpp embeddings. This can also be checked in the next section where we use "Agent memory" as query for both retrievers.

In [ ]:
embd_model_path = os.path.join(os.getcwd(), "llama.cpp", "model", "nomic-embed-text-v1.Q2_K.gguf")
llama_embedding = LlamaCppEmbeddings(model_path=embd_model_path, n_batch=512)

# Index
llama_vectorstore = Chroma.from_documents(
    documents= all_splits,
    collection_name= "rag-chroma-llama",
    embedding= embedding,
)

llama_retriever= llama_vectorstore.as_retriever()

llama_model_loader: loaded meta data with 23 key-value pairs and 112 tensors from /content/llama.cpp/model/nomic-embed-text-v1.Q2_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = nomic-bert
llama_model_loader: - kv   1:                               general.name str              = nomic-embed-text-v1
llama_model_loader: - kv   2:                     nomic-bert.block_count u32              = 12
llama_model_loader: - kv   3:                  nomic-bert.context_length u32              = 2048
llama_model_loader: - kv   4:                nomic-bert.embedding_length u32              = 768
llama_model_loader: - kv   5:             nomic-bert.feed_forward_length u32              = 3072
llama_model_loader: - kv   6:            nomic-bert.attention.head_count u32              = 12
llama_model_loader: - kv   7:    nomic-bert

## Retrieve example document to check if collection has data

In [ ]:
for d in retriever.get_relevant_documents("Agent memory"):
    print("=" * 40)
    print(d.page_content)

Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.

Each element is an observation, an event directly provided by the agent.
- Inter-agent communication can trigger new natural language statements.


Retrieval model: surfaces the context to inform the agent’s behavior, according to relevance, recency and importance.

Recency: recent events have higher scores
Importance: distinguish mundane from core memories. Ask LM directly.
Relevance: based on how related it is to the current situation / query.


Reflection mechanism: synthesizes memories into higher level inferences over time and guides the agent’s future behavior. They are higher-level summaries of past events (<- note that this is a bit different from self-reflection above)

Prompt LM with 100 most recent observations and to generate 3 most salient high-level questions given a set of observations/statements. Then ask LM to answer those questi

## Llama | Retrieve example document to check if collection has data (Optional)

In [ ]:
for d in llama_retriever.get_relevant_documents("Agent memory"):
    print("=" * 40)
    print(d.page_content)

Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.

Each element is an observation, an event directly provided by the agent.
- Inter-agent communication can trigger new natural language statements.


Retrieval model: surfaces the context to inform the agent’s behavior, according to relevance, recency and importance.

Recency: recent events have higher scores
Importance: distinguish mundane from core memories. Ask LM directly.
Relevance: based on how related it is to the current situation / query.


Reflection mechanism: synthesizes memories into higher level inferences over time and guides the agent’s future behavior. They are higher-level summaries of past events (<- note that this is a bit different from self-reflection above)

Prompt LM with 100 most recent observations and to generate 3 most salient high-level questions given a set of observations/statements. Then ask LM to answer those questi

# Start async process to start `ollama serve`

This will start the `ollama serve` command which is necessary to run `ollama run mistral:instruct`

In [ ]:
import os
import asyncio

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

In [ ]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

>>> starting ollama serve
time=2024-03-01T09:53:45.885Z level=INFO source=images.go:710 msg="total blobs: 5"
time=2024-03-01T09:53:45.885Z level=INFO source=images.go:717 msg="total unused blobs removed: 0"
time=2024-03-01T09:53:45.886Z level=INFO source=routes.go:1019 msg="Listening on 127.0.0.1:11434 (version 0.1.27)"
time=2024-03-01T09:53:45.886Z level=INFO source=payload_common.go:107 msg="Extracting dynamic libraries..."


# Simple chain to use retriever

- This chain will take a promt template to instruct llm that it is a grader and need to output json as its response.

- The json output will then pass through a JsonOutputParser which will parse the string json to python native dict format.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser

# LLM
llm= ChatOllama(model= local_llm, format="json", temperature= 0)

prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n
    Here is the retrieved document: \n\n {context} \n\n
    Here is the user question: {question} \n
    If the document contains keywords related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination.""",
    input_variables=["question", "context"],
)

chain = prompt | llm | JsonOutputParser()

question = "Explain how the different types of agent memory work?"
docs= retriever.get_relevant_documents(question)
score = chain.invoke({"question": question, "context": docs[0].page_content})
print(score)

time=2024-03-01T06:16:44.280Z level=INFO source=payload_common.go:146 msg="Dynamic LLM libraries [rocm_v6 rocm_v5 cpu cpu_avx cpu_avx2 cuda_v11]"
time=2024-03-01T06:16:44.280Z level=INFO source=gpu.go:94 msg="Detecting GPU type"
time=2024-03-01T06:16:44.280Z level=INFO source=gpu.go:265 msg="Searching for GPU management library libnvidia-ml.so"
time=2024-03-01T06:16:44.285Z level=INFO source=gpu.go:311 msg="Discovered GPU libraries: [/usr/lib64-nvidia/libnvidia-ml.so.535.104.05]"
time=2024-03-01T06:16:44.294Z level=INFO source=gpu.go:99 msg="Nvidia GPU detected"
time=2024-03-01T06:16:44.294Z level=INFO source=cpu_common.go:11 msg="CPU has AVX2"
time=2024-03-01T06:16:44.300Z level=INFO source=gpu.go:146 msg="CUDA Compute Capability detected: 7.5"
time=2024-03-01T06:16:44.601Z level=INFO source=cpu_common.go:11 msg="CPU has AVX2"
time=2024-03-01T06:16:44.602Z level=INFO source=gpu.go:146 msg="CUDA Compute Capability detected: 7.5"
time=2024-03-01T06:16:44.602Z level=INFO source=cpu_commo

# State

Every node in our graph will modify `state`, which is dict that contains values (`question`, `documents`, etc) relevant to RAG.

In [ ]:
from typing import Annotated, Dict, TypedDict

from langchain_core.messages import BaseMessage


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

# Nodes and Edges

Every node in the graph we laid out above is a function.

Each node will modify the state in some way.

Each edge will choose which node to call next.

In [ ]:
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_community.chat_models import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mistralai.chat_models import ChatMistralAI

### Nodes ###


def dprint(*args)->None:
    for txt in args:
        print("-" * 20 + f" {txt} " + "-" * 20)

def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    local = state_dict["local"]
    retriever_to_use = state_dict['retriever_to_use']
    dprint(retriever_to_use, "GPT4All using= " + str(retriever_to_use == "GPT4All"))
    documents = retriever.get_relevant_documents(question) if retriever_to_use == "GPT4All" else llama_retriever.get_relevant_documents(question)
    return {"keys": {"documents": documents, "local": local, "question": question}}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains generation
    """
    print("---GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    local = state_dict["local"]

    # Prompt
    prompt = hub.pull("rlm/rag-prompt")

    # LLM
    if local == "Yes":
        llm = ChatOllama(model=local_llm, temperature=0)
    else:
        llm = ChatMistralAI(
            model="mistral-medium", temperature=0, mistral_api_key=mistral_api_key
        )

    # Post-processing
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {
        "keys": {"documents": documents, "question": question, "generation": generation}
    }


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with relevant documents
    """

    print("---CHECK RELEVANCE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    local = state_dict["local"]

    # LLM
    if local == "Yes":
        llm = ChatOllama(model=local_llm, format="json", temperature=0)
    else:
        llm = ChatMistralAI(
            mistral_api_key=mistral_api_key, temperature=0, model="mistral-medium"
        )

    prompt = PromptTemplate(
        template="""You are a grader assessing relevance of a retrieved document to a user question. \n
        Here is the retrieved document: \n\n {context} \n\n
        Here is the user question: {question} \n
        If the document contains keywords related to the user question, grade it as relevant. \n
        It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
        Provide the binary score as a JSON with a single key 'score' and no premable or explaination.""",
        input_variables=["question", "context"],
    )

    chain = prompt | llm | JsonOutputParser()

    # Score
    filtered_docs = []
    search = "No"  # Default do not opt for web search to supplement retrieval
    for d in documents:
        score = chain.invoke(
            {
                "question": question,
                "context": d.page_content,
            }
        )
        grade = score["score"]
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            search = "Yes"  # Perform web search
            continue

    return {
        "keys": {
            "documents": filtered_docs,
            "question": question,
            "local": local,
            "run_web_search": search,
        }
    }


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    local = state_dict["local"]

    # Create a prompt template with format instructions and the query
    prompt = PromptTemplate(
        template="""You are generating questions that is well optimized for retrieval. \n
        Look at the input and try to reason about the underlying sematic intent / meaning. \n
        Here is the initial question:
        \n ------- \n
        {question}
        \n ------- \n
        Provide an improved question without any premable, only respond with the updated question: """,
        input_variables=["question"],
    )

    # Grader
    # LLM
    if local == "Yes":
        llm = ChatOllama(model=local_llm, temperature=0)
    else:
        llm = ChatMistralAI(
            mistral_api_key=mistral_api_key, temperature=0, model="mistral-medium"
        )

    # Prompt
    chain = prompt | llm | StrOutputParser()
    better_question = chain.invoke({"question": question})

    return {
        "keys": {"documents": documents, "question": better_question, "local": local}
    }


def web_search(state):
    """
    Web search based on the re-phrased question using Tavily API.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Web results appended to documents.
    """

    print("---WEB SEARCH---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    local = state_dict["local"]

    tool = TavilySearchResults()
    docs = tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results) # append web documents to retrieved documents

    return {"keys": {"documents": documents, "local": local, "question": question}}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer or re-generate a question for web search.

    Args:
        state (dict): The current state of the agent, including all keys.

    Returns:
        str: Next node to call
    """

    print("---DECIDE TO GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    filtered_documents = state_dict["documents"]
    search = state_dict["run_web_search"]

    if search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: TRANSFORM QUERY and RUN WEB SEARCH---")
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

# Build Graph

This just follows the flow we outlined in the figure above.

In [ ]:
import pprint

from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query
workflow.add_node("web_search", web_search)  # web search

# Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "web_search")
workflow.add_edge("web_search", "generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

# Run

## GPT4All generation

In [ ]:
# Run
inputs = {
    "keys": {
        "question": "Explain how the different types of agent memory work?",
        "local": run_local,
        "retriever_to_use" : "GPT4All"
    }
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

# Final generation
pprint.pprint(value["keys"]["generation"])

---RETRIEVE---
-------------------- GPT4All --------------------
-------------------- GPT4All using= True --------------------
"Node 'retrieve':"
'\n---\n'
---CHECK RELEVANCE---
[GIN] 2024/03/01 - 06:26:23 | 200 |  1.196520008s |       127.0.0.1 | POST     "/api/chat"
---GRADE: DOCUMENT RELEVANT---
[GIN] 2024/03/01 - 06:26:25 | 200 |  1.511803217s |       127.0.0.1 | POST     "/api/chat"
---GRADE: DOCUMENT RELEVANT---
[GIN] 2024/03/01 - 06:26:26 | 200 |  1.372822226s |       127.0.0.1 | POST     "/api/chat"
---GRADE: DOCUMENT RELEVANT---
[GIN] 2024/03/01 - 06:26:27 | 200 |  643.753474ms |       127.0.0.1 | POST     "/api/chat"
---GRADE: DOCUMENT RELEVANT---
"Node 'grade_documents':"
'\n---\n'
---DECIDE TO GENERATE---
---DECISION: GENERATE---
---GENERATE---
[GIN] 2024/03/01 - 06:26:38 | 200 |  9.875794888s |       127.0.0.1 | POST     "/api/chat"
"Node 'generate':"
'\n---\n'
"Node '__end__':"
'\n---\n'
(' In an LLM (large language model)-powered autonomous agent system, LLM '
 'function

## Llama.cpp generation

In [ ]:
# Run
inputs = {
    "keys": {
        "question": "Explain how the different types of agent memory work?",
        "local": run_local,
        "retriever_to_use" : "Llama"
    }
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

# Final generation
pprint.pprint(value["keys"]["generation"])

---RETRIEVE---
-------------------- Llama --------------------
-------------------- GPT4All using= False --------------------
"Node 'retrieve':"
'\n---\n'
---CHECK RELEVANCE---
[GIN] 2024/03/01 - 06:26:48 | 200 |  1.186081081s |       127.0.0.1 | POST     "/api/chat"
---GRADE: DOCUMENT RELEVANT---
[GIN] 2024/03/01 - 06:26:49 | 200 |  1.523827574s |       127.0.0.1 | POST     "/api/chat"
---GRADE: DOCUMENT RELEVANT---
[GIN] 2024/03/01 - 06:26:51 | 200 |  1.413219356s |       127.0.0.1 | POST     "/api/chat"
---GRADE: DOCUMENT RELEVANT---
[GIN] 2024/03/01 - 06:26:51 | 200 |   646.22748ms |       127.0.0.1 | POST     "/api/chat"
---GRADE: DOCUMENT RELEVANT---
"Node 'grade_documents':"
'\n---\n'
---DECIDE TO GENERATE---
---DECISION: GENERATE---
---GENERATE---
[GIN] 2024/03/01 - 06:27:02 | 200 |  9.379334629s |       127.0.0.1 | POST     "/api/chat"
"Node 'generate':"
'\n---\n'
"Node '__end__':"
'\n---\n'
(' In an LLM (large language model)-powered autonomous agent system, LLM '
 'functions

# Creating a server with google colab for local computer access

## Install dependencies

- aiohttp
- pyngrok

In [ ]:
!pip install aiohttp pyngrok

## NGrok access token

1. Create an ngrok account from [here](https://ngrok.com/).

2. Get [ngrok auth token from here](https://dashboard.ngrok.com/get-started/your-authtoken).

In [ ]:
NGROK_ACCESS_TOKEN = '<NGROK_ACCESS_TOKEN>'

## Start async processes

In [ ]:
import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

#register an account at ngrok.com and create an authtoken and place it here
await asyncio.gather(
    run_process(['ngrok', 'config', 'add-authtoken', NGROK_ACCESS_TOKEN])
)

await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ngrok', 'http', '--log', 'stderr', '11434']),
)


>>> starting ngrok config add-authtoken 2d4vI2QK5xk1qcer128P4Q1E6EJ_2DKznD7bSFCsAqmGk5T9S
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
>>> starting ollama serve
>>> starting ngrok http --log stderr 11434
Error: listen tcp 127.0.0.1:11434: bind: address already in use
t=2024-03-01T09:54:17+0000 lvl=info msg="no configuration paths supplied"
t=2024-03-01T09:54:17+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
t=2024-03-01T09:54:17+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
t=2024-03-01T09:54:17+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
t=2024-03-01T09:54:18+0000 lvl=info msg="client session established" obj=tunnels.session obj=csess id=3be6455cdd9c
t=2024-03-01T09:54:18+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2024-03-01T09:54:18+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localho

## Connecting from Local PC

The below code provided is for connecting from your local device via ngrok exposed url.


- Once the ngrok service has been started then copy the below code for connecting with google colab using local device

```python
#! Python 3.8.10
import requests
import json
import argparse


# Example URL to make a GET request
NGROK_EXPOSED_URL = "<NGROK_EXPOSED_URL>"
OLLAMA_URL = "http://127.0.0.1:11434/api/generate"
LOCAL_LLM = "mistral:instruct"


def parse_arguments():
    parser = argparse.ArgumentParser(description="Process some integers.")

    # Define command-line arguments with default values
    parser.add_argument(
        "--model", type=str, default="mistral:instruct", help="model name eg. mistral"
    )
    parser.add_argument("--url", type=str, help="url of exposed api")
    parser.add_argument(
        "--forever", action="store_true", help="run only one time or in loop"
    )
    parser.add_argument(
        "--stream", action="store_false", help="output from url will be streamed"
    )

    # Parse the arguments
    args = parser.parse_args()

    # Convert parsed arguments into a dictionary
    args_dict = vars(args)

    # Filter out arguments with default values
    args_dict = {k: v for k, v in args_dict.items() if v is not None}

    return args_dict


def ollama_generate_from_url(
    user_query: str, url: str = OLLAMA_URL, model=LOCAL_LLM, stream=True
):
    print("*" * 30 + " Configs " + "*" * 30)
    print(f"{user_query = }")
    print(f"{url = }")
    print(f"{model = }")
    print(f"{stream = }")
    print("*" * 30 + "  " + "*" * 30)

    payload = {
        "model": model,  # Your model name
        "prompt": user_query,  # prompt for the model
    }

    try:
        # Making a GET request with stream=True
        response = requests.post(url, json=payload, stream=stream)

        # Iterating over the response content line by line
        for line in response.iter_lines():
            if line:
                # Process each line of the streaming response
                response_dict = json.loads(line.decode("utf-8"))
                print(response_dict["response"], end="")  # Decode bytes to string
    except requests.RequestException as e:
        print("Error occurred:", e)


def start_ollama_server(
    model: str = LOCAL_LLM, forever=False, url: str = OLLAMA_URL, stream=True
):

    user_query = input(f"\nAsk ollama: ")
    ollama_generate_from_url(model=model, url=url, user_query=user_query, stream=stream)

    while forever:
        user_query = input(f"Ask ollama: ")
        ollama_generate_from_url(
            model=model, url=url, user_query=user_query, stream=stream
        )


def main():
    args = parse_arguments()
    start_ollama_server(**args)


if __name__ == "__main__":
    main()

```


- Save this code as `ollama-colab.py`.

- From terminal __run the below command__

    `python ollama-colab.py --url <ngrok-exposed-url>`

- Running the ollama locally can also be done with this script. If the url is not provided it will try to access local device localhost for generation at the port 11434. Run the below command for local ollama access

    ```python ollama-colab.py```
    or
    ```python ollama-colab.py --url http://127.0.0.1:11434/api/generate```


- __Other optional args__.

    - model (str): The default model will be `mistral:instruct`. Provide any other model name to generate response.

    ```python ollama-colab.py --model gemma```

    - stream (boolean): This will use the streaming for the post request on ollama host.

    ```python ollama-colab.py --stream```

    - forever (boolean): This will run an infinite while loop for chat like feel.


# References:

- Ollama api docs: https://github.com/ollama/ollama/blob/main/docs/api.md
- Langchain corrective rag: https://www.youtube.com/watch?v=E2shqsYwxck